In [1]:
import requests
import pandas as pd
import os

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go

In [10]:
data = pd.read_csv("../mydata/cleandata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)

In [12]:
data.dtypes

product     object
price      float64
dtype: object

In [3]:
def toprophet(food):
    fooddicc = {}
    date = []
    price = []
    for i, row in data.iterrows():
        if row['product'] == f'{food}':
            date.append(i)
            price.append(row['price'])

        fooddicc['product'] = f'{food}'
        fooddicc['ds'] = date
        fooddicc['y'] = price
    foodframe = pd.DataFrame(fooddicc)
    foodframe.drop(['product'], axis=1, inplace=True) # Deleting columns product and date to prepare them to plot a graph
    
    groupframe = pd.DataFrame(foodframe.groupby(foodframe.ds)["y"].mean()) # Some dates are repeated, so I group by date
    groupframe.sort_values(by='ds')
    groupframe = groupframe.reset_index()
    groupframe ['ds'] = pd.to_datetime(groupframe.ds)
    return groupframe

In [4]:
for i in set(data['product']):
    prophet = toprophet(f'{i}')
    if not os.path.exists("../mydata/per_item/for_prophet"):
        os.makedirs("../mydata/per_item/for_prophet")
    prophet.to_csv(f"../mydata/per_item/for_prophet/prophet_{i}.csv",index=False)